****Домашнее задание****

In [13]:
import platform
if platform.system() == 'Linux':
    path = '/content/drive/My Drive/Colab Notebooks/netology_pyda/'
    from google.colab import drive
    drive.mount('/content/drive')
else:
    path = ''

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [6]:
def rate_level(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating >= 4.5:
        return 'высокий рейтинг'
    else:
        return 'средний рейтинг'
    
import pandas as pd
data = pd.read_csv(path +'ratings.csv')
data['class'] = data.rating.apply(rate_level)
data

,userId,movieId,rating,timestamp,class
0,1,296,5.0,1147880044,высокий рейтинг
1,1,306,3.5,1147868817,средний рейтинг
2,1,307,5.0,1147868828,высокий рейтинг
3,1,665,5.0,1147878820,высокий рейтинг
4,1,899,3.5,1147868510,средний рейтинг
...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,высокий рейтинг
25000091,162541,55768,2.5,1240951998,средний рейтинг
25000092,162541,56176,2.0,1240950697,низкий рейтинг
25000093,162541,58559,4.0,1240953434,средний рейтинг


**Задание 2**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [14]:
def region(x):
    geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
                'Северо-Запад': ['петербург', 'псков', 'мурманск'],
                'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}
    x = set(x.lower().split(' '))
    for i in geo_data:
        if not x.isdisjoint(set(geo_data[i])):
            return i
    return 'undefined'
        
key = pd.read_csv(path + 'keywords.csv')
key['region'] = key.keyword.apply(region)

key2 = key.groupby('region').count()
print(key2, '\n\n', key[key.region != 'undefined'])

                keyword  shows
region                        
undefined         99406  99406
Дальний Восток       64     64
Северо-Запад        146    146
Центр               384    384 

                                      keyword   shows          region
127                             авито москва  979292           Центр
370                 авито ру санкт петербург  425134    Северо-Запад
849                          авито ярославль  209581           Центр
1063                     фарпост владивосток  176951  Дальний Восток
1236                    банк санкт петербург  174375    Северо-Запад
...                                      ...     ...             ...
99292           афиша кино москва на сегодня    3704           Центр
99590  авито ярославль автомобили с пробегом    3673           Центр
99808      полармед мурманск запись на прием    3719    Северо-Запад
99890                    яндекс метро москва    4548           Центр
99922                      авто ру ярославль    3712

**Задание 3 (бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [15]:
def production_year(x):
    years = list(range(1950, 2011))
    for i in years:
        if str(i) in str(x):
            return str(i)
    return str(1900)

ratings = pd.read_csv(path + 'ratings.csv')
movies = pd.read_csv(path + 'movies.csv')
tab = ratings.merge(movies, left_on='movieId', right_on='movieId')

h = 1000000 # мой ноутбук слишком долго обрабатывает всю таблицу, поэтому только 1000000 строк
tab['year'] = tab.head(h).title.apply(production_year)
tab.head(h).groupby('year').mean().sort_values(by = 'rating', ascending=False)

,userId,movieId,rating,timestamp
year,,,,
1994,79356.503739,302.929692,4.178228,1.149018e+09
1900,80963.563819,1497.398371,4.149363,1.172819e+09
1985,80666.209195,1217.000000,4.142050,1.143061e+09
1977,81554.041926,260.000000,4.120189,1.195385e+09
1957,81165.761923,1706.892588,4.094601,1.160762e+09
1966,80370.143889,7327.000000,4.093056,1.360953e+09
2002,81166.069310,5983.796516,4.084375,1.321318e+09
1952,80788.553373,899.000000,4.050987,1.148433e+09
1954,80220.591304,7234.000000,4.010870,1.290439e+09
